In [11]:
from model import model_init, BraggNN_MLP
import torch, argparse, os, time, sys, shutil, logging
from util import str2bool, str2tuple, s2ituple
from torch.utils.data import DataLoader
from dataset import BraggNNDataset
import numpy as np
from torchsummary import summary

In [12]:
torch_devs = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
itr_out_dir = 'debug'
if os.path.isdir(itr_out_dir): 
    shutil.rmtree(itr_out_dir)
os.mkdir(itr_out_dir) # to save temp output

In [14]:
numLayers = 2
numNeurons = 2
fcsz = [numNeurons*100 for i in range(1,numLayers)]
    
model = BraggNN_MLP(fcsz=fcsz)
print(summary(model))

def main():
    logging.info("[%.3f] loading data into CPU memory, it will take a while ... ..." % (time.time(), ))
    ds_train = BraggNNDataset(psz=11, rnd_shift=1, use='train')
    dl_train = DataLoader(dataset=ds_train, batch_size=512, shuffle=True,\
                          num_workers=8, prefetch_factor=512, drop_last=True, pin_memory=True)

    ds_valid = BraggNNDataset(psz=11, rnd_shift=0, use='validation')
    dl_valid = DataLoader(dataset=ds_valid, batch_size=512, shuffle=False, \
                          num_workers=8, prefetch_factor=512, drop_last=False, pin_memory=True)


    _ = model.apply(model_init) # init model weights and bias


    
    if torch.cuda.is_available():
        gpus = torch.cuda.device_count()
        if gpus > 1:
            logging.info("This implementation only makes use of one GPU although %d are visiable" % gpus)
        model = model.to(torch_devs)

    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.002) 
    
    for epoch in range(200):
        for X_mb, y_mb in dl_train:

            optimizer.zero_grad()
            pred = model.forward(X_mb.to(torch_devs))
            loss = criterion(pred, y_mb.to(torch_devs))
            loss.backward()
            optimizer.step() 

        pred_val, gt_val = [], []
        for X_mb_val, y_mb_val in dl_valid:
            with torch.no_grad():
                _pred = model.forward(X_mb_val.to(torch_devs))
                pred_val.append(_pred.cpu().numpy())
                gt_val.append(y_mb_val.numpy())
        pred_val = np.concatenate(pred_val, axis=0)
        gt_val   = np.concatenate(gt_val,   axis=0)

        pred_train = pred.cpu().detach().numpy()  
        true_train = y_mb.cpu().numpy()  
        l2norm_train = np.sqrt((true_train[:,0] - pred_train[:,0])**2   + (true_train[:,1] - pred_train[:,1])**2) * 11
        l2norm_val   = np.sqrt((gt_val[:,0]     - pred_val[:,0])**2     + (gt_val[:,1]     - pred_val[:,1])**2)   * 11

        print('[Train] @ %05d l2-norm of %5d samples: Avg.: %.4f, 50th: %.3f, 75th: %.3f, 95th: %.3f, 99.5th: %.3f (pixels).' % (\
                     (epoch, l2norm_train.shape[0], l2norm_train.mean()) + tuple(np.percentile(l2norm_train, (50, 75, 95, 99.5))) ) )

        print('[Valid] @ %05d l2-norm of %5d samples: Avg.: %.4f, 50th: %.3f, 75th: %.3f, 95th: %.3f, 99.5th: %.3f (pixels) \n' % (\
                     (epoch, l2norm_val.shape[0], l2norm_val.mean()) + tuple(np.percentile(l2norm_val, (50, 75, 95, 99.5))) ) )

    torch.save(model.state_dict(), "final.pth")

Layer (type:depth-idx)                   Param #
├─Sequential: 1-1                        --
|    └─Linear: 2-1                       24,400
|    └─LeakyReLU: 2-2                    --
|    └─Linear: 2-3                       402
Total params: 24,802
Trainable params: 24,802
Non-trainable params: 0
Layer (type:depth-idx)                   Param #
├─Sequential: 1-1                        --
|    └─Linear: 2-1                       24,400
|    └─LeakyReLU: 2-2                    --
|    └─Linear: 2-3                       402
Total params: 24,802
Trainable params: 24,802
Non-trainable params: 0


In [7]:
if __name__ == "__main__":
    main()

[Train] @ 00000 l2-norm of   512 samples: Avg.: 0.5610, 50th: 0.480, 75th: 0.745, 95th: 1.265, 99.5th: 2.064 (pixels).
[Valid] @ 00000 l2-norm of 13799 samples: Avg.: 0.5855, 50th: 0.500, 75th: 0.778, 95th: 1.327, 99.5th: 2.128 (pixels) 

[Train] @ 00001 l2-norm of   512 samples: Avg.: 0.4215, 50th: 0.337, 75th: 0.551, 95th: 1.066, 99.5th: 1.489 (pixels).
[Valid] @ 00001 l2-norm of 13799 samples: Avg.: 0.4336, 50th: 0.351, 75th: 0.574, 95th: 1.038, 99.5th: 1.565 (pixels) 

[Train] @ 00002 l2-norm of   512 samples: Avg.: 0.3822, 50th: 0.282, 75th: 0.512, 95th: 0.986, 99.5th: 1.308 (pixels).
[Valid] @ 00002 l2-norm of 13799 samples: Avg.: 0.3872, 50th: 0.287, 75th: 0.528, 95th: 0.985, 99.5th: 1.445 (pixels) 

[Train] @ 00003 l2-norm of   512 samples: Avg.: 0.3359, 50th: 0.250, 75th: 0.454, 95th: 0.853, 99.5th: 1.170 (pixels).
[Valid] @ 00003 l2-norm of 13799 samples: Avg.: 0.3492, 50th: 0.257, 75th: 0.474, 95th: 0.911, 99.5th: 1.331 (pixels) 

[Train] @ 00004 l2-norm of   512 samples: Av